In [2]:
import pandas as pd
import numpy as np
import os as os
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import funcionalidade as fc

In [3]:
inativos = pd.read_pickle('inativos_fatura_premiada.pkl')

In [3]:
#inativos.to_csv('inativos_fatura_premiada.csv')

In [4]:
inativos['grupo_comunicacao'].value_counts()

Comunicação MCC         732744
Comunicação Genérica     95054
Sem comunicação          88540
Name: grupo_comunicacao, dtype: int64

In [4]:
#inativos['fx_ie'] = pd.qcut(inativos['nr_score_ie'].rank(method='first'), q=3, precision=0, labels = ['baixo', 'medio', 'alto'])
#inativos[['fx_ie', 'nr_score_ie']].groupby('fx_ie').agg({'nr_score_ie':['min','max']})

nr_score_ie           
              min        max
fx_ie                       
baixo    0.000000   2.968275
medio    2.968275  13.815732
alto    13.815732  95.567145

## INATIVOS - Análise comunicação genérica x MCC

In [5]:
inativos['grupo'] = ['alvo' if x != 'Sem comunicação' else 'controle' for x in inativos['grupo_comunicacao']]

### Trazendo conversão

In [6]:
inativos_c = fc.traz_conversao('flavia-costa', '2023-03-17', '2023-03-24', 'cpf', inativos)

In [7]:
len(inativos_c)

916338

In [8]:
inativos_c['converteu'].value_counts()

0.0    854299
1.0     62039
Name: converteu, dtype: int64

### Traz informações complememtares - personas e recência

In [9]:
import acessos as ac

In [10]:
query = """
    select *, 
    date_diff('day', cast(ultima_data as date), to_date('2023-03-17', 'yyyy-mm-dd')) as recencia
    from (
    select ci.cpf
    , max(ds_cluster_sociodemographic) as ds_cluster_sociodemographic
    , max(sp.dt_dia) as ultima_data
    from customer_curated_zone.ca_book_cliente_cluster cl
    left join customer_curated_zone.ca_book_cliente ci on (ci.id_customer = cl.id_customer)
    left join platform_curated_zone.transaction_dedicada_spending sp on (sp.cpf = ci.cpf and sp.dt_data < to_date('2023-03-17','yyyy-mm-dd'))
    group by 1
)"""

In [11]:
inativos_c_p = inativos_c.join(ac.df_athena_q('flavia-costa', query).set_index('cpf'), how = 'left', on = 'cpf')

In [12]:
#Missings
inativos_c_p['recencia'].fillna(inativos_c_p['recencia'].max(), inplace = True)
inativos_c_p['ds_cluster_sociodemographic'].fillna(inativos_c_p['ds_cluster_sociodemographic'].mode()[0], inplace = True)

In [13]:
inativos_c_p['recencia'].describe()

count    916338.000000
mean       1123.781880
std         610.388949
min           1.000000
25%         455.000000
50%        1536.000000
75%        1536.000000
max        1536.000000
Name: recencia, dtype: float64

In [14]:
inativos_c_p['fx_recencia'] = np.where(inativos_c_p['recencia'] <= 60, 1,
                             np.where(inativos_c_p['recencia'] <= 120, 2,
                                       np.where(inativos_c_p['recencia'] <= 180, 3,
                                            np.where(inativos_c_p['recencia'] <= 360, 4
                                                ,5))))

In [15]:
inativos_c_p['fx_recencia'].value_counts()

5    704404
1     74097
4     54070
2     45182
3     38585
Name: fx_recencia, dtype: int64

### Análises de Lift

Análise reduzida geral

In [16]:
fc.testes_iterativos(inativos_c)

           grupo converteu    spending              transacoes          
           count      mean        mean          sum       mean       sum
grupo                                                                   
alvo      827798  0.067462  384.784448  21488287.48   2.836422  158400.0
controle   88540  0.069957  390.021319   2415792.05   2.829835   17528.0
_____________________
Lift da conversão: -3.57%
Estatística t do teste de diferença de proporções: -2.81
P-valor do teste de diferença de proporções: 0.004976
Conclusão:
A diferença de conversão é estatisticamente significativa
______________________________________________________________
Lift do spending: -1.34%
Estatística t do teste de spending: -1.64
P-valor do teste de spending: 0.100096
Conclusão:
A diferença de spending não é significativa
______________________________________________________________
Lift da frequência: 0.23%
Estatística t do teste de frequencia: -1.77
P-valor do teste de frequencia: 0.076805
A diferen

Análise por recortes

In [17]:
#cria faixa de IE
inativos_c['fx_ie'] = pd.qcut(inativos_c['nr_score_ie'].rank(method='first'), q=3, precision=0, labels = ['baixo', 'medio', 'alto'])
inativos_c['fx_ie'].value_counts()

baixo    97160
medio    97160
alto     97160
Name: fx_ie, dtype: int64

In [18]:
for x in ['baixo', 'medio', 'alto']:
    temp_df = pd.DataFrame()
    print(f"=============Faixa de IE: {x}============")
    temp_df = inativos_c[inativos_c['fx_ie'] == x]
    fc.testes_iterativos(temp_df)

=============Faixa de IE: baixo============
          grupo converteu    spending            transacoes        
          count      mean        mean        sum       mean     sum
grupo                                                              
alvo      88264  0.013981  469.350308  579178.28   2.820908  3481.0
controle   8896  0.013939  350.376371   43446.67   2.637097   327.0
_____________________
Lift da conversão: 0.3%
Estatística t do teste de diferença de proporções: 0.03
P-valor do teste de diferença de proporções: 0.974382
Conclusão:
A diferença de conversão não é significativa
______________________________________________________________
Lift do spending: 33.96%
Estatística t do teste de spending: 1.31
P-valor do teste de spending: 0.189481
Conclusão:
A diferença de spending não é significativa
______________________________________________________________
Lift da frequência: 6.97%
Estatística t do teste de frequencia: 0.48
P-valor do teste de frequencia: 0.629306
A difere

In [19]:
inativos_c.segmento_mcc_agg.value_counts()

0 - OUTROS                     914635
5 - LOJAS ESPECIALIZADAS          520
3 - SERVICOS DE ALIMENTACAO       345
2 - LOJAS DE DEPARTAMENTO         299
4 - SAUDE                         272
1 - SUPERMERCADO                  267
Name: segmento_mcc_agg, dtype: int64

In [20]:
for x in inativos_c_p.fx_recencia.unique():
    temp_df = pd.DataFrame()
    print(f"=============Faixa Recencia: {x}============")
    temp_df = inativos_c_p[inativos_c_p['fx_recencia'] == x]
    fc.testes_iterativos(temp_df)

=============Faixa Recencia: 1============
          grupo converteu    spending             transacoes         
          count      mean        mean         sum       mean      sum
grupo                                                                
alvo      66760  0.414679  268.260074  7426511.88   3.285905  90967.0
controle   7337  0.415429  261.399600   796745.98   3.287730  10021.0
_____________________
Lift da conversão: -0.18%
Estatística t do teste de diferença de proporções: -0.12
P-valor do teste de diferença de proporções: 0.901604
Conclusão:
A diferença de conversão não é significativa
______________________________________________________________
Lift do spending: 2.62%
Estatística t do teste de spending: 0.57
P-valor do teste de spending: 0.569292
Conclusão:
A diferença de spending não é significativa
______________________________________________________________
Lift da frequência: -0.06%
Estatística t do teste de frequencia: -0.1
P-valor do teste de frequencia: 0.923

In [21]:
#por segmento MCC
for x in ['0 - OUTROS', '4 - SAUDE', '3 - SERVICOS DE ALIMENTACAO','2 - LOJAS DE DEPARTAMENTO','1 - SUPERMERCADO', '5 - LOJAS ESPECIALIZADAS']:
    temp_df = pd.DataFrame()
    print(f"=============Segmento MCC: {x}============")
    temp_df = inativos_c[inativos_c['segmento_mcc_agg'] == x]
    fc.testes_iterativos(temp_df)

=============Segmento MCC: 0 - OUTROS============
           grupo converteu    spending              transacoes          
           count      mean        mean          sum       mean       sum
grupo                                                                   
alvo      826268  0.067107  384.208059  21303568.45   2.835359  157215.0
controle   88367  0.069472  389.799785   2392980.88   2.827008   17355.0
_____________________
Lift da conversão: -3.4%
Estatística t do teste de diferença de proporções: -2.67
P-valor do teste de diferença de proporções: 0.007664
Conclusão:
A diferença de conversão é estatisticamente significativa
______________________________________________________________
Lift do spending: -1.43%
Estatística t do teste de spending: -1.61
P-valor do teste de spending: 0.107131
Conclusão:
A diferença de spending não é significativa
______________________________________________________________
Lift da frequência: 0.3%
Estatística t do teste de frequencia: -1.64
P-

In [22]:
personas_list = inativos_c_p['ds_cluster_sociodemographic'].unique()
print(personas_list)

['17 - Homem, 25 a 40 anos, Sudeste e Sul'
 '0 - Mulher, Casada, 25 anos ou mais, cidade Pequena do Nordeste'
 '11 - Homem, Solteiro, Até 24 anos, cidade Pequena do Nordeste'
 '20 - Homem, Até 24 anos, Sudeste e Sul'
 '24 - Homem, Até 24 anos, Nordeste'
 '18 - Mulher, 25 a 40 anos, Sudeste e Sul'
 '16 - Homem, 25 a 40 anos, Nordeste'
 '10 - Mulher, Solteira, Até 24 anos, cidade Pequena do Nordeste'
 '25 - Homem, 41 anos ou mais, Sudeste e Sul'
 '14 - Homem, Casado, 35 anos ou mais, cidade Pequena do Nordeste'
 '27 - Homem, 25 a 40 anos, Centro-Oeste e Norte'
 '15 - Mulher, 25 a 40 anos, Nordeste'
 '23 - Homem, 41 anos ou mais, Nordeste'
 '31 - Mulher, 41 anos ou mais, Centro-Oeste e Norte'
 '8 - Mulher, Solteira, Até 24 anos, cidade Pequena do Sudeste ou Sul'
 '21 - Mulher, Até 24 anos, Nordeste'
 '26 - Mulher, 25 a 40 anos, Centro-Oeste e Norte'
 '29 - Mulher, Até 24 anos, Sudeste e Sul'
 '1 - Mulher, Solteira, Até 24 anos, cidade Média do Sudeste ou Sul'
 '2 - Mulher, Solteira, 35 an

In [23]:
for x in personas_list:
    temp_df = pd.DataFrame()
    print(f"=============Personas: {x}============")
    if x is None:
        pass
    else:
        temp_df = inativos_c_p[inativos_c_p['ds_cluster_sociodemographic'] == x]
        fc.testes_iterativos(temp_df)

=============Personas: 17 - Homem, 25 a 40 anos, Sudeste e Sul============
          grupo converteu    spending             transacoes         
          count      mean        mean         sum       mean      sum
grupo                                                                
alvo      80070  0.041127  553.805381  1823681.12   3.184330  10486.0
controle   8260  0.046126  545.199974   207721.19   3.052493   1163.0
_____________________
Lift da conversão: -10.84%
Estatística t do teste de diferença de proporções: -2.17
P-valor do teste de diferença de proporções: 0.030258
Conclusão:
A diferença de conversão é estatisticamente significativa
______________________________________________________________
Lift do spending: 1.58%
Estatística t do teste de spending: -0.85
P-valor do teste de spending: 0.396457
Conclusão:
A diferença de spending não é significativa
______________________________________________________________
Lift da frequência: 4.32%
Estatística t do teste de frequenc

## Analise de combinações

In [24]:
import importlib
importlib.reload(fc)

<module 'funcionalidade' from '/home/flavia.costa/work/codigos/FaturaPremiadaMar2023/funcionalidade.py'>

In [25]:
inativos_c_p['cont'] = 1

In [26]:
colunas_analise = ['ds_cluster_sociodemographic', 'fx_ie', 'fx_recencia']

inativos_c_p['fx_ie'] = inativos_c['fx_ie'].astype('string')

report_combinacoes_alvo = fc.report_perfil_grupos(inativos_c_p.query('grupo == "alvo"'), 'converteu', 'cont', 0.05, 50, colunas_analise)
report_combinacoes_alvo.set_index('segmento', inplace=True)

report_combinacoes_controle = fc.report_perfil_grupos(inativos_c_p.query('grupo == "controle"'), 'converteu', 'cont', 0.05, 50, colunas_analise)
report_combinacoes_controle.set_index('segmento', inplace=True)

#.query('grupo_comunicacao ==  "Comunicação MCC"')


In [27]:
report_combinacoes = report_combinacoes_alvo.join(report_combinacoes_controle, rsuffix='_c', lsuffix='_t')

In [28]:
report_combinacoes.head()

,n_seg_grupo_t,n_seg_t,n_grupo_t,total_t,per_grupo_seg_t,per_grupo_total_t,diferenca_t,diferenca_abs_t,converteu_t,n_seg_grupo_c,n_seg_c,n_grupo_c,total_c,per_grupo_seg_c,per_grupo_total_c,diferenca_c,diferenca_abs_c,converteu_c
segmento,,,,,,,,,,,,,,,,,,
"- 0 Mulher, Casada, 25 anos ou mais, cidade Pequena do Nordeste alto",566,4289,55845,827798,0.131965,0.067462,0.064503,0.064503,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"- 0 Mulher, Casada, 25 anos ou mais, cidade Pequena do Nordeste alto",3723,4289,771953,827798,0.868035,0.932538,-0.064503,0.064503,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"- 0 Mulher, Casada, 25 anos ou mais, cidade Pequena do Nordeste alto 1",546,723,771953,827798,0.755187,0.932538,-0.177351,0.177351,0.0,55,82,82346,88540,0.670732,0.930043,-0.259311,0.259311,0.0
"- 0 Mulher, Casada, 25 anos ou mais, cidade Pequena do Nordeste alto 1",177,723,55845,827798,0.244813,0.067462,0.177351,0.177351,1.0,55,82,82346,88540,0.670732,0.930043,-0.259311,0.259311,0.0
"- 0 Mulher, Casada, 25 anos ou mais, cidade Pequena do Nordeste alto 2",254,1691,55845,827798,0.150207,0.067462,0.082745,0.082745,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
report_combinacoes.query('converteu_t == 1').query('converteu_c == 1').query('diferenca_t > 0').query('diferenca_t - diferenca_c > 0.04').sort_values('diferenca_t', ascending=False)
#.to_csv('temp.csv')


,n_seg_grupo_t,n_seg_t,n_grupo_t,total_t,per_grupo_seg_t,per_grupo_total_t,diferenca_t,diferenca_abs_t,converteu_t,n_seg_grupo_c,n_seg_c,n_grupo_c,total_c,per_grupo_seg_c,per_grupo_total_c,diferenca_c,diferenca_abs_c,converteu_c
segmento,,,,,,,,,,,,,,,,,,


In [30]:
temp_df = inativos_c_p[inativos_c_p['ds_cluster_sociodemographic'].isna() == False] \
        .query('ds_cluster_sociodemographic.str.contains("18 - Mulher, 25 a 40 anos, Sudeste")')
        
fc.testes_iterativos(temp_df)

          grupo converteu    spending             transacoes         
          count      mean        mean         sum       mean      sum
grupo                                                                
alvo      70923  0.053988  441.433056  1690247.17   3.050927  11682.0
controle   7669  0.058547  434.695835   195178.43   3.155902   1417.0
_____________________
Lift da conversão: -7.79%
Estatística t do teste de diferença de proporções: -1.67
P-valor do teste de diferença de proporções: 0.09456
Conclusão:
A diferença de conversão não é significativa
______________________________________________________________
Lift do spending: 1.55%
Estatística t do teste de spending: -0.63
P-valor do teste de spending: 0.529264
Conclusão:
A diferença de spending não é significativa
______________________________________________________________
Lift da frequência: -3.33%
Estatística t do teste de frequencia: -1.63
P-valor do teste de frequencia: 0.103402
A diferença de frequência não é signif